In [ ]:
import os

from weco_datascience.reporting import get_recent_data

In [ ]:
df = get_recent_data(config=os.environ, n=10000,
                     index="metrics-conversion-prod")


In [ ]:
# grab only the fields needed

keepers = [
    "@timestamp",
    "anonymousId",
    "page.name",
    "page.query.workId",
    "page.query.canvas",
    "page.query.manifest",
]
df = df[keepers]
df.head()

In [ ]:
# what time period does this start from?
df = df.sort_values("@timestamp")
df.head()

In [ ]:
# How many users view non-images

users = df.loc[(df["page.name"] == "item")]
users2 = users.sort_values("anonymousId")
users2.nunique()
# users3=users2.drop_duplicates(subset=['anonymousId'], keep=False)

# users3['anonymousId'].count()

# so 12,781 users viewed non-image items

In [ ]:
# How many users use the manifest button

clickers = df.loc[(df["page.query.manifest"].notnull())]
clickers.nunique()

# so 119 users looked at 76 works and used manifest nav
# possible to view pages without using manifest

In [ ]:
# success = manifest>0, canvas>1

viewers = df.loc[
    (df["page.query.manifest"].notnull()) & (df["page.query.canvas"] != "1")
]
viewers.nunique()

# so 71 users used the manifest nav for 64 unique workIds and got past page 1

In [ ]:
# how many users viewed works with manifests whether or not they used the nav

# create lookup table for workIds with manifests
workId_only = ["page.query.workId"]
works_lookup = clickers[workId_only]
works_lookup2 = works_lookup.sort_values("page.query.workId")  # 76 unique
works_lookup2.drop_duplicates(inplace=True)

works_w_manifests_accessed = works_lookup2.merge(df, how="left", on="page.query.workId")
works_w_manifests_accessed.nunique()

# so 258 users accessed works with manifests. Out of 119 manifest nav users, only 71 got past page 1,
# 139 users didn't use the manifest navigation at all. 68 used it but couldn't get past page 1.